Back to the main [Index](../index.ipynb)

# Post-processing tools for phonons

Is this tutorial, we will explore the tools provided by Abipy for the post-processing of DFPT calculations. Let's start by importing the basic modules we have already used in the previous lessons:


In [3]:
from __future__ import division, print_function, unicode_literals

import abipy.data as abidata

from abipy import abilab
from abipy.abilab import abiopen

# This line configures matplotlib to show figures embedded in the notebook, 
# instead of poping up a new window. 
%matplotlib notebook

# Reading a DDB file in python

In [7]:
ddb_path = "/Users/gmatteo/git_repos/abipy/abipy/test_files/AlAs_444_nobecs_DDB"
ddb = abiopen(ddb_path)
# Remember to close the ddb with ddb.close() when you don't need it anymore!

###### The DDB file has a header, a dict-like object with the most important parameters of the calculation

In [8]:
# To sort the keys in the header and print the list use:
from pprint import pprint
pprint(sorted(list(ddb.header.keys())))

[u'acell',
 u'amu',
 u'dilatmx',
 u'ecut',
 u'ecutsm',
 u'intxc',
 u'iscf',
 u'ixc',
 u'kpt',
 u'kptnrm',
 'lines',
 u'natom',
 u'nband',
 u'ngfft',
 u'nkpt',
 u'nspden',
 u'nspinor',
 u'nsppol',
 u'nsym',
 u'ntypat',
 u'occ',
 u'occopt',
 u'rprim',
 u'sciss',
 u'spinat',
 u'symafm',
 u'symrel',
 u'tnons',
 u'tolwfr',
 u'tphysel',
 u'tsmear',
 u'typat',
 u'usepaw',
 'version',
 u'wtk',
 u'xred',
 u'zion',
 u'znucl']


In [9]:
# Use the stadard syntax [] to access the value of a variable in the header.
print("This DDB has been generated with ecut", ddb.header["ecut"], "and amu", ddb.header["amu"])

This DDB has been generated with ecut 2.0 and amu [26.981539, 74.92159]


###### To get the list of $q$-points available in the DDB file:

In [10]:
ddb.qpoints

0) [0.000, 0.000, 0.000]
1) [0.250, 0.000, 0.000]
2) [0.500, 0.000, 0.000]
3) [0.250, 0.250, 0.000]
4) [0.500, 0.250, 0.000]
5) [-0.250, 0.250, 0.000]
6) [0.500, 0.500, 0.000]
7) [-0.250, 0.500, 0.250]

###### A DDB object has a crystalline structure

In [11]:
# Remember that lengths are always given in Angstrom.
ddb.structure

Structure Summary
Lattice
    abc : 3.9701006655510169 3.9701006655510169 3.9701006655510169
 angles : 59.999999999999993 59.999999999999993 59.999999999999993
 volume : 44.247583621484289
      A : 0.0 2.8072851026043497 2.8072851026043497
      B : 2.8072851026043497 0.0 2.8072851026043497
      C : 2.8072851026043497 2.8072851026043497 0.0
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: As (1.4036, 1.4036, 1.4036) [0.2500, 0.2500, 0.2500]

# Invoking Anaddb form the DDB object 

The `DdbFile` object provides specialized methods that will invoke anaddb to 
compute important quantities such as the phonon band structure, the phonon density of states etc.
All these methods have a name that begins with the `ana` prefix

###### Plotting phonon bands and DOSes 

In [12]:
ddb.anaget_phbands_and_dos(ngqpt=None, ndivsm=20, nqsmall=10, 
                           asr=2, chneut=1, dipdip=1, dos_method="tetra")

AttributeError: 'DdbFile' object has no attribute 'anaget_phbands_and_dos'

###### Computing DOSes with different $q$-meshes 

The method `anaconverge_phdos` provides a simple interface that allows us to compare phonon DOSes
computed with different $q$-meshes. In many cases, you only have to provide a 
list of integers (`nqsmalls`). Each integer defines the number of divisions to be used to 
sample the smallest reciprocal lattice vectors, the other two vectors are sampled such 
that proportions are preserver. 
To calculate four phonon DOSes with increasing number of q-points use:

In [13]:
c = ddb.anaconverge_phdos(nqsmalls=[2, 4, 8, 12, 24])
figure = c.plotter.plot()

AttributeError: 'DdbFile' object has no attribute 'anaconverge_phdos'

Anaddb requires the specification of the q-mesh used for the phonons.
The DDB object will try to figure out the number of divisions in the 
mesh from ddb.qpoints.
This information is used to generate automatically the Anaddb input file
Note, however, that if `guessed_ngqpt` is wrong, you will have to pass
this value to all the ddb methods that invoke anaddb.
This could happen if you have merged DDB files computed with $q$-points that do not belong to same grid. 

In [14]:
ddb.guessed_ngqpt

array([4, 4, 4])

# Macroscopic dielectric tensor and Born effective charges (BECs)

In [15]:
emacro, becs = ddb.anaget_emacro_and_becs(chneut=1)

In [16]:
emacro

(Tensor in r space.
 
 Cartesian coordinates:
 [[  1.00000000e+00   1.11022302e-16   1.11022302e-16]
  [  1.11022302e-16   1.00000000e+00   1.11022302e-16]
  [  1.11022302e-16   1.11022302e-16   1.00000000e+00]],)

What's happening here? According to anaddb, AlAs should have a dielectric 
constant of one and this is cleary wrong!
Remember that the computation macroscopic dielectric requires the knowledge 
of $\dfrac{d u_k}{d k}$ (the so-called DDK files)

# Using `DdbRobot` to perform converge studies

Robots are extremely useful if you want to study the convergence of the phonon frequencies at with respect to some computational parameter e.g. `ecut`. 
A `DddRobot` receives a list of DDB files, extracts the data and provides tools to analyze 
the convergence or low-level tools that are just returning a `Pandas` dataframe. 

In [ ]:
files = [
        "flow_qconv/w1/outdata/out_DDB",
        "flow_qconv/w3/outdata/out_DDB",
        "flow_qconv/w5/outdata/out_DDB",
        "flow_qconv/w7/outdata/out_DDB",
        "flow_qconv/w9/outdata/out_DDB",
    ]
from abipy.abio.robots import DdbRobot
robot = DdbRobot()
for i, f in enumerate(files):
    robot.add_file(str(i), f) 
print(robot)

frame = robot.get_dataframe_at_qpoint()
print(frame)

robot.plot_conv_phfreqs_qpoint(x_vars=["ecut"], size=1, aspect=.5)

# Low-level interface

In [ ]:
# This example shows how to plot the phonon band structure of AlAs.
# See tutorial/lesson_rf2.html

# FIXME: LO-TO splitting and phonon displacements instead of eigenvectors.
#from abipy.dfpt.phonons import PhononBands, PhdosReader, PhdosFile

# Path to the PHBST file produced by anaddb.
phbst_file = abidata.ref_file("trf2_5.out_PHBST.nc")

# Create the object from file.
with abilab.abiopen(phbst_file) as f:
    phbands = f.phbands

# Read the Phonon DOS from the netcd file produced by anaddb (prtdos 2)
phdos_file = abidata.ref_file("trf2_5.out_PHDOS.nc")

with abiopen(phdos_file) as f:
    phdos = f.phdos

# plot phonon bands and DOS.
figure = phbands.plot_with_phdos(phdos, title="AlAs Phonon bands and DOS")

In [ ]:
# This example shows how to plot the phonon fatbands of AlAs.
# See tutorial/lesson_rf2.html

# Path to the PHBST file produced by anaddb.
filename = abidata.ref_file("trf2_5.out_PHBST.nc")
with abiopen(filename) as f:
    phbands = f.phbands 

# Mapping reduced coordinates -> labels
qlabels = {
    (0,0,0): "$\Gamma$",
    (0.375, 0.375, 0.75): "K",
    (0.5, 0.5, 1.0): "X",
    (0.5, 0.5, 0.5): "L",
    (0.5, 0.0, 0.5): "X",
    (0.5, 0.25, 0.75): "W",
}

# Plot the phonon band structure.
figure = phbands.plot_fatbands(title="AlAs phonon fatbands without LO-TO splitting", 
                               qlabels=qlabels)

In [ ]:
# This example shows how to plot the projected phonon DOS of AlAs.
# See tutorial/lesson_rf2.html

# Read the Phonon DOS from the netcd file produced by anaddb (prtdos 2)
phdos_file = abiopen(abidata.ref_file("trf2_5.out_PHDOS.nc"))

# Plot data.
figure = phdos_file.plot_pjdos_type(title="AlAs type-projected phonon DOS")